In [2]:
import numpy as np
import pandas as pd

In [3]:
url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv"
df = pd.read_csv(url)

In [4]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   object 
 1   industry                  1328 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1281 non-null   float64
 4   employment_status         1362 non-null   object 
 5   location                  1399 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [6]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [102]:
cat = list(df.columns[df.dtypes == 'object'])
print(cat)
num = list(df.columns[df.dtypes != 'object'])
print(num)
num_w_target = num[:4]
print(num_w_target)

['lead_source', 'industry', 'employment_status', 'location']
['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score', 'converted']
['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']


In [81]:
for c in cat:
    df[c] = df[c].fillna('NA')
for n in num:
    df[n] = df[n].fillna(0.0)

Q1

In [82]:
df['industry'].value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

Q2

In [83]:
df[num].corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879,0.435914
annual_income,0.009770,1.000000,0.027036,0.015610,0.053131
interaction_count,-0.023565,0.027036,1.000000,0.009888,0.374573
lead_score,-0.004879,0.015610,0.009888,1.000000,0.193673
converted,0.435914,0.053131,0.374573,0.193673,1.000000


In [84]:
from sklearn.model_selection import train_test_split

Q3

In [85]:
X = df.drop('converted', axis=1)
y = df['converted']

In [86]:
y.shape

(1462,)

In [87]:
X.shape
y.shape

(1462,)

In [117]:
X_full_train, X_test, y_full_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_full_train, y_full_train, test_size=0.25, random_state=42)

In [118]:
X_test.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score
892,social_media,manufacturing,2,56070.0,self_employed,middle_east,2,0.23
1106,NA,other,1,78409.0,NA,australia,4,0.79
413,referral,manufacturing,2,66206.0,employed,australia,3,0.30
522,events,retail,0,0.0,self_employed,north_america,2,0.98
1036,organic_search,retail,6,62832.0,unemployed,NA,4,1.00


In [119]:
y_test.head()

892     1
1106    0
413     1
522     0
1036    1
Name: converted, dtype: int64

In [120]:
from sklearn.metrics import mutual_info_score

In [121]:
cat

['lead_source', 'industry', 'employment_status', 'location']

In [122]:
for c in cat:
    print(f'{c}: {round(mutual_info_score(X_train[c].values, y_train.values),2)}')

lead_source: 0.04
industry: 0.01
employment_status: 0.01
location: 0.0


Q4

In [123]:
from sklearn.linear_model import LogisticRegression

In [124]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

In [125]:
dv = DictVectorizer(sparse=False)

In [126]:
train_dict = X_train[cat + num_w_target].to_dict(orient= 'records')
X_train = dv.fit_transform(train_dict)

In [127]:
val_dict = X_val[cat + num_w_target].to_dict(orient= 'records')
X_val = dv.transform(val_dict)

In [109]:
X_val

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score
662,paid_ads,healthcare,3,52220.0,unemployed,europe,1,0.07
600,organic_search,technology,3,59656.0,unemployed,middle_east,4,0.65
477,events,manufacturing,0,57134.0,self_employed,north_america,4,0.13
1057,events,other,0,0.0,NA,asia,0,0.03
891,referral,retail,1,54103.0,unemployed,south_america,3,0.16
...,...,...,...,...,...,...,...,...
1367,social_media,healthcare,1,55222.0,self_employed,africa,1,0.25
1390,paid_ads,NA,1,20326.0,employed,middle_east,3,0.81
419,organic_search,technology,1,74166.0,employed,south_america,2,0.01
114,organic_search,technology,2,39103.0,self_employed,africa,3,0.60


(293, 31)

In [159]:
model = LogisticRegression(solver='liblinear', C=0.1, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,0.1
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [160]:
y_pred = model.predict_proba(X_val)[:,1]
y_pred_2 = model.predict(X_val)

In [161]:
converted_decision = y_pred >= 0.5

In [162]:
(y_val == converted_decision).mean()

0.6996587030716723

In [163]:
from sklearn.metrics import accuracy_score

In [158]:
round(accuracy_score(y_pred_2, y_val), 3)

0.7

Q6

In [144]:
original_acc = 0.7

In [165]:
C = [0.01, 0.1, 1, 10, 100, 50000]

In [166]:
for c in C:
    model = LogisticRegression(solver='liblinear', C=c)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    acc = accuracy_score(y_pred, y_val)
    print(f'{c}: {acc}')


0.01: 0.6996587030716723
0.1: 0.6996587030716723
1: 0.6996587030716723
10: 0.6996587030716723
100: 0.6996587030716723
50000: 0.6996587030716723
